In [ ]:
from keras.applications.densenet import DenseNet121
base_model = DenseNet121(include_top=True, weights='imagenet')
base_model.summary()

Using TensorFlow backend.


33193984/33188688 [==============================] - 14s 0us/step


In [ ]:
import numpy as np
import pandas as pd
import os
import h5py
from keras.utils.io_utils import HDF5Matrix

In [ ]:
h5_path = '../chest_xray.h5'
disease_vec_labels = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion',
                      'Emphysema','Fibrosis','Hernia','Infiltration','Mass','Nodule',
                      'Pleural_Thickening','Pneumonia','Pneumothorax']
disease_vec = []
with h5py.File(h5_path, 'r') as h5_data:
    all_fields = list(h5_data.keys())
    for c_key in all_fields:
        print(c_key, h5_data[c_key].shape, h5_data[c_key].dtype)
    for c_key in disease_vec_labels:
        disease_vec += [h5_data[c_key][:]]
disease_vec = np.stack(disease_vec,1)
print('Disease Vec:', disease_vec.shape)

In [ ]:
# generate binary label
binary_vec = [0 if label.sum() == 0 else 1 for label in disease_vec]
binary_vec = np.asarray(binary_vec)
print(binary_vec.shape)

In [ ]:
img_ds = HDF5Matrix(h5_path, 'images')
split_idx = int(img_ds.shape[0] * 0.9)
X_train = HDF5Matrix(h5_path, 'images', end=split_idx)
X_test = HDF5Matrix(h5_path, 'images', start=split_idx)
y_train = binary_vec[:split_idx]
y_test = binary_vec[split_idx:]
print('Train Shape', X_train.shape, 'test shape', X_test.shape)

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import GlobalAveragePooling2D, AveragePooling2D, GlobalMaxPool2D, Dense, Dropout, Flatten, Conv2D
from keras.models import Sequential
from keras.utils import multi_gpu_model

In [ ]:
nb_records, nb_classes = binary_vec.shape
img_shape = img_ds.shape[1:]
nb_gpus = 2

base_model = DenseNet121(input_shape=img_shape, include_top=False, weights='imagenet')
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model = multi_gpu_model(model, gpus=nb_gpus)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path="weights_{epoch:02d}_{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)
early = EarlyStopping(monitor="loss", mode="min", patience=3)

In [ ]:
batch_size = 64
nb_epoch = 50

model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1,
          validation_split=0.2,
          callbacks=[checkpoint, early]
          )

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Predicting")
y_pred = model.predict(X_test)
print(y_test.shape, y_pred.shape)

In [ ]:
# y_test = np.argmax(y_test, axis=1)
# y_pred = np.argmax(y_pred, axis=1)

precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average="weighted")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

In [ ]:
model.save('./model-20181012.h5')